```{contents}
```
## TransformChain (LangChain)


**TransformChain** is a **legacy LangChain abstraction** used to **apply a custom Python transformation function** to inputs (and optionally outputs) **inside a chain pipeline**.

> It allows you to inject **deterministic, non-LLM logic** into an LLM workflow.

TransformChain does **not call an LLM** by itself.

---

### Why TransformChain Exists

TransformChain was created to:

* Preprocess inputs before an LLM call
* Postprocess outputs after an LLM call
* Glue multiple chains together
* Apply deterministic business logic

Examples:

* Text normalization
* Metadata enrichment
* Field extraction
* Validation
* Formatting

---

### Core Idea

```
Input (dict)
   ↓
Python Transform Function
   ↓
Output (dict)
```

The function receives a dictionary and returns a dictionary.

---

### Basic TransformChain Demonstration

#### Step 1: Define a Transform Function



In [1]:
def add_length(inputs: dict) -> dict:
    text = inputs["input"]
    return {
        "input": text,
        "length": len(text)
    }




---

#### Step 2: Create a TransformChain



In [3]:
from langchain_classic.chains import TransformChain

transform_chain = TransformChain(
    input_variables=["input"],
    output_variables=["input", "length"],
    transform=add_length
)




---

#### Step 3: Invoke the Chain



In [4]:

result = transform_chain.invoke({"input": "LangChain"})
print(result)


{'input': 'LangChain', 'length': 9}




Output:

```python
{
  "input": "LangChain",
  "length": 9
}
```

---

### TransformChain with LLMChain

#### Preprocessing Before LLM Call



In [5]:
from langchain_classic.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = PromptTemplate(
    input_variables=["input", "length"],
    template="Text: {input}\nLength: {length}\nExplain the text."
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

pipeline = transform_chain | llm_chain


C:\Users\sangouda\AppData\Local\Temp\ipykernel_25864\2931510592.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)




---

#### Invocation



In [6]:
pipeline.invoke({"input": "LangChain"})


{'input': 'LangChain',
 'length': 9,
 'text': '"LangChain" is a framework designed for building applications that utilize large language models (LLMs). It provides tools and components that facilitate the integration of LLMs with various data sources, APIs, and other functionalities, enabling developers to create more complex and interactive applications. LangChain can be used for tasks such as chatbots, data analysis, and content generation, among others. Its modular design allows for flexibility and customization, making it easier for developers to leverage the power of language models in their projects.'}



The transform enriches input before the LLM runs.

---

### TransformChain as Post-Processor

#### Cleaning LLM Output



In [7]:
def clean_output(inputs: dict) -> dict:
    return {
        "clean_text": inputs["text"].strip().lower()
    }
post_process_chain = TransformChain(
    input_variables=["text"],
    output_variables=["clean_text"],
    transform=clean_output
)




Used after an LLMChain.

---

### TransformChain vs RunnableLambda (Modern)

#### TransformChain (Legacy)



In [8]:
def my_function(inputs: dict) -> dict:
    """Example transform function for TransformChain."""
    return {
        "output": inputs.get("input", "").upper()
    }

In [10]:
TransformChain(
    input_variables=["input"],
    output_variables=["output"],
    transform=my_function
)


TransformChain(verbose=False, input_variables=['input'], output_variables=['output'], transform_cb=<function my_function at 0x0000021061BFA980>)



### RunnableLambda (Recommended)

```python
from langchain_core.runnables import RunnableLambda

transform = RunnableLambda(my_function)
```

RunnableLambda:

* Is Runnable-native
* Supports streaming & async
* Works with LCEL
* Future-proof

---

### Limitations of TransformChain

* ❌ Legacy abstraction
* ❌ Not fully Runnable-based
* ❌ Limited async support
* ❌ No streaming
* ❌ No parallelism

---

### When TransformChain Is Still Used

* Maintaining old LangChain projects
* Legacy tutorials
* Quick glue logic in older chains

---

### When NOT to Use TransformChain

* New projects
* LCEL-based pipelines
* Agents
* RAG systems
* Production APIs

Use **RunnableLambda** instead.

---

### Migration Example (TransformChain → RunnableLambda)

### Before

```python
TransformChain(transform=fn)
```

### After

```python
RunnableLambda(fn)
```

Same logic, better execution model.

---

### Best Practices (If You Must Use It)

* Keep transforms deterministic
* Avoid heavy computation
* Validate inputs explicitly
* Plan migration to LCEL

---

### Interview-Ready Summary

> “TransformChain is a legacy LangChain construct for inserting custom Python transformations into a chain pipeline. It has been superseded by RunnableLambda, which integrates natively with LCEL and supports modern execution features.”

---

### Rule of Thumb

* **Legacy chain → TransformChain**
* **Modern pipeline → RunnableLambda**
* **Any new system → LCEL only**

---

If you want next:

* RunnableLambda deep dive
* TransformChain vs SequentialChain
* LCEL glue patterns
* Migrating legacy chains to LCEL
